# Problem Set 2
Name: Jeremy Tan <br>
Date: March 8, 2024

## Problem 2A

In [1]:
P = [] # list of points (x-coor,y-coor)

with open("seg_ls.txt", "r") as file:
    n = file.readline()
    for line in file:
        x,y = line.split(" ")
        x = float(x)
        y = float(y)
        P.append([x,y])


# We have to sort by increasing x-coordinate
sorted_points = sorted(P, key=lambda P: P[0])

In [2]:
n = 1000 # we have 1000 points
points = sorted_points
# Precompute the prefix sums
P_sum_x = [0]
P_sum_y = [0]
P_sum_x2 = [0]
P_sum_y2 = [0]
P_sum_xy = [0]

# The prefix sum arrays start at index 1 until index n
for i in range(1, n+1):
    P_sum_x.append(P_sum_x[i-1] + points[i-1][0])
    P_sum_y.append(P_sum_y[i-1] + points[i-1][1])
    P_sum_x2.append(P_sum_x2[i-1] + (points[i-1][0])**2)
    P_sum_y2.append(P_sum_y2[i-1] + (points[i-1][1])**2)
    P_sum_xy.append(P_sum_xy[i-1] + (points[i-1][0]*points[i-1][1]))
    

In [4]:
def partial_sum(arr, i, j):
    return arr[j] - arr[i-1]  

def L(i, j):
    """
  Calculates the least squares regression coefficients for a set of points.

  Args:
      i - left index 
      j - right index

  Returns:
      Slope - slope of the least squares line
      Intercept - intercept of the least squares line
      Error - sum of squared errors after performing simple linear regression
    """
    n = j - i + 1 # number of points
    # solve for the slope and y-intercept
    slope = ((n*partial_sum(P_sum_xy, i, j)) - (partial_sum(P_sum_x, i,j)*partial_sum(P_sum_y, i,j))) / ((n*partial_sum(P_sum_x2, i,j)) - (partial_sum(P_sum_x,i,j))**2)
    intercept = (partial_sum(P_sum_y, i,j) - slope*partial_sum(P_sum_x,i,j))/n
    
    # Calculate total error (sum of squared residuals)
    error = n*(intercept**2) + 2*intercept*slope*partial_sum(P_sum_x,i,j) + (slope**2)*partial_sum(P_sum_x2,i,j) - 2*intercept*partial_sum(P_sum_y,i,j) - 2*slope*partial_sum(P_sum_xy,i,j) + partial_sum(P_sum_y2,i,j)
    return slope, intercept, error

def least_squares(j,k):
    if (j,k) in memo:
        return memo[j,k]
    else: 
        if j == 0:
            minimum_error = 0
            
        # Only one line left
        elif k == 1:
            slope, intercept, minimum_error = L(1,j)
        else:
            minimum_error = float('inf')
            for i in range(2*k - 1, j):
                slope, intercept, error = L(i, j)
                penalty = error + least_squares(i-1, k-1)
                # Get the minimum error
                if penalty < minimum_error:
                    minimum_error = penalty
                    min_i = i
                    # Save the partition of the best fitting line
                partitions[j,k] = min_i 

        memo[j,k] = minimum_error
        
    return memo[j,k]
                
            


memo = dict()

partitions = dict() # used to store the 

print(least_squares(n,3)) # set k = 3 lines



294090.4364505969


In [5]:
# We have to find the point i for the first least squares line:
print(partitions[1000,3]) # output is i = 677
# The first least squares line goes from i = 677 to j = 1000

# For the next line, we look at partitions[i-1, k-1] = partitions[676, 2]
print(partitions[676,2]) # output is i = 324

# For the third (last) line, we already know that it is from i = 1 to j = 323


677
324


In [6]:
print('Least Squares Lines:')
slope_1, intercept_1, error_1 = L(677, 1000)
slope_2, intercept_2, error_2 = L(324, 676)
slope_3, intercept_3, error_3 = L(1, 323)

# We reorder the line numbering so the first line starts with i = 1
print(f'The first line is y = {slope_3}x + {intercept_3} with endpoints: {points[0]} to {points[322]}')
print(f'The second line is y = {slope_2}x + {intercept_2} with endpoints: {points[323]} to {points[675]}')
print(f'The third line is y = {slope_1}x + {intercept_1} with endpoints: {points[676]} to {points[999]}')

Least Squares Lines:
The first line is y = 2.6198719328315083x + 41.41697391132595 with endpoints: [0.599, 8.2329] to [168.6967, 446.1948]
The second line is y = -0.08277141390232377x + 499.52630522715805 with endpoints: [170.4067, 447.1715] to [341.5237, 435.9841]
The third line is y = -2.7978472945909423x + 1428.2028072016205 with endpoints: [343.1398, 441.3405] to [498.5574, -0.9299]


## Problem 2b

In [149]:
def least_squares_arbitary(C):
    # base case
    f[0] = 0

    for j in range(1, n+1):
        minimum_error = float('inf')

        # Find the minimum penalty per j
        for i in range(1,j):
            slope, intercept, error = L(i, j)
            penalty = error + C + f[i-1]

            if penalty <  minimum_error:
                minimum_error = penalty

                # Given j find the optimal i
                partitions2[j] = i

        f[j] = minimum_error
    return f[j]

f = [None] * (n+1)
partitions2 = dict() # save the optimal i for each j

C = 10000 # cost per line
min_error = least_squares_arbitary(C)
print(min_error)

100109.20639497414


In [157]:
# Similar to 2a we check the partitions2 dictionary to find the partitions of the least squares lines

print(partitions2[1000]) # First Line - output is i = 841
print(partitions2[840]) # Second Line - output is i = 677
print(partitions2[676]) # Third Line - output is i = 497
print(partitions2[496]) # Fourth Line - output is i = 313
print(partitions2[312]) # Fifth Line - output is i = 157
print(partitions2[156]) # Sixth Line - output is i = 1


841
677
497
313
157
1


In [159]:
print('Least Squares Lines:')
slope_1, intercept_1, error_1 = L(841, 1000)
slope_2, intercept_2, error_2 = L(677, 840)
slope_3, intercept_3, error_3 = L(497, 676)
slope_4, intercept_4, error_4 = L(313, 496)
slope_5, intercept_5, error_5 = L(157, 312)
slope_6, intercept_6, error_6 = L(1, 156)

# We reorder the line numbering so the first line starts with i = 1
print(f'The first line is y = {slope_6}x + {intercept_6} with endpoints: {points[0]} to {points[155]}')
print(f'The second line is y = {slope_5}x + {intercept_5} with endpoints: {points[156]} to {points[311]}')
print(f'The third line is y = {slope_4}x + {intercept_4} with endpoints: {points[312]} to {points[495]}')
print(f'The fourth line is y = {slope_3}x + {intercept_3} with endpoints: {points[496]} to {points[675]}')
print(f'The fifth line is y = {slope_2}x + {intercept_2} with endpoints: {points[676]} to {points[839]}')
print(f'The sixth line is y = {slope_1}x + {intercept_1} with endpoints: {points[840]} to {points[999]}')

Least Squares Lines:
The first line is y = 3.2698572772406687x + 11.47512592259059 with endpoints: [0.599, 8.2329] to [86.9667, 294.2807]
The second line is y = 1.975571957171812x + 125.49428628367815 with endpoints: [88.7357, 287.2646] to [163.5283, 445.1173]
The third line is y = 0.7051310552970829x + 334.1239676269403 with endpoints: [164.7958, 433.1884] to [250.3958, 499.4984]
The fourth line is y = -0.767198281558675x + 704.0776964268915 with endpoints: [252.0358, 505.7027] to [341.5237, 435.9841]
The fifth line is y = -2.1598034478458956x + 1182.461368158822 with endpoints: [343.1398, 441.3405] to [421.9327, 265.6377]
The sixth line is y = -3.3706682936738726x + 1693.969139518941 with endpoints: [422.7515, 262.1774] to [498.5574, -0.9299]


## Problem 3

In [52]:
# a) Suppose that there are 3 containers. We set k = 3. 

'''
        n (int): Number of trash bags
        M (int): Maximum weight capacity of each container
        w (list): List of weights of trash bags
        k (int): Number of containers available
'''

n = int(input())
M = int(input())
w = [0] + [int(x) for x in input().split()]
k = 3


# Initially, all three containers have available capacity of M.

c1 = M # remaining capacity of container 1
c2 = M # remaining capacity of container 2
c3 = M # remaining capacity of container 3

memo = dict()

def top_down_dp(i, c1, c2, c3):
# ci is the remaining capacity in the ith container
    
    
    # Define three boolean variables that state if it is possible to add trash bag i to the container
    
    add1 = False
    add2 = False
    add3 = False
    key = (i, c1, c2, c3)
    if key in memo:
        return memo[key]
    else: 
        # One Trash Bag Left
        if i == 1:
        # The answer is true if at least one container has enough capacity to hold the last trash bag
            if (w[i] <= c1 and c1 >= 0) or (w[i] <= c2 and c2 >= 0) or (w[i] <= c3 and c3 >= 0):
                answer = True
        # The answer is false if none of the containers have enough capacity to hold the last trash bag
            else:
                answer = False
        # We have three different cases, if the weight of bag i is less than or equal to the available capacity in container 1, container 2, and container 3
        else:
            if (w[i] <= c1 and c1 >= 0) or (w[i] <= c2 and c2 >= 0) or (w[i] <= c3 and c3 >= 0):
                
                if (w[i] <= c1 and c1 >= 0):
                    add1 = top_down_dp(i - 1, c1 - w[i], c2, c3)
                
                if (w[i] <= c2 and c2 >=0):
                    add2 = top_down_dp(i -1 , c1, c2 - w[i], c3)
                
                if (w[i] <= c3 and c3 >=0):
                    add3 = top_down_dp(i -1 , c1, c2, c3- w[i])
                    
                answer = add1 or add2 or add3
        # If the weight of trash bag i is over the remaining capacity in all three containers, we output false
            else:
                answer = False
    memo[key] = answer
    return memo[key]

    
flag = 0
for bag in w:
    if bag > M:
        flag = 1
        print(False)

if flag == 0:
    answer = top_down_dp(n, c1, c2, c3)
    if answer == True:
        print(f'The answer is True. It is possible to fit all the trash bags into 3 containers.')
    if answer == False:
        print(f'The answer is False. It is not possible to fit all the trash bags into 3 containers.')


 8 
 13
  3 2 12 7 1 6 2 6


The answer is True. It is possible to fit all the trash bags into 3 containers.


In [54]:
# b) Suppose that there are 2 containers. We set k = 2. 

'''
        n (int): Number of trash bags
        M (int): Maximum weight capacity of each container
        w (list): List of weights of trash bags
        k (int): Number of containers available
'''

n = int(input())
M = int(input())
w = [0] + [int(x) for x in input().split()]
k = 2

# Initially, both containers have available capacity of M.

c1 = M # remaining capacity of container 1
c2 = M # remaining capacity of container 2

memo = dict()

def top_down_dp(i, c1, c2):
# ci is the remaining capacity in the ith container

    # Define two boolean variables that state if it is possible to add trash bag i to the container
    
    add1 = False
    add2 = False
    
    key = (i, c1, c2)
    if key in memo:
        return memo[key]
    else: 
        # We consider the case when there is only one bag left to add to the containers
        if i == 1:
        # The answer is true if at least one container has enough capacity to hold the last trash bag
            if (w[i] <= c1 and c1 >= 0) or (w[i] <= c2 and c2 >= 0):
                answer = True
        # The answer is false if none of the containers have enough capacity to hold the last trash bag
            else:
                answer = False
        # We have two cases, if the weight of bag i is less than or equal to the available capacity in container 1 or container 2
        else:
            if (w[i] <= c1 and c1 >= 0) or (w[i] <= c2 and c2 >= 0):
                if w[i] <= c1 and c1 >= 0:
                    add1 = top_down_dp(i - 1, c1 - w[i], c2) 
                if w[i] <= c2 and c2 >= 0:
                    add2 = top_down_dp(i - 1, c1, c2 - w[i])
                answer = add1 or add2
        # If the weight of trash bag i is over the remaining capacity in both containers, we output false
            else:
                answer = False
    memo[key] = answer
    return memo[key]
    
flag = 0
for bag in w:
    if bag > M:
        flag = 1
        print(False)

if flag == 0:
    result = top_down_dp(n, c1, c2)
    if result == True:
        print(f'The answer is True. It is possible to fit all the trash bags into 2 containers.')
    if result == False:
        print(f'The answer is False. It is not possible to fit all the trash bags into 2 containers.')



 8
 11
 2 3 1 2 5 3 5 1


The answer is True. It is possible to fit all the trash bags into 2 containers.


## Problem 4

In [68]:
def max_subarray_sum(R):
    """
    Finds the maximum subarray sum using top down Dynamic Programming.

    Args:
        R: right index 

    Returns:
        answer: the maximum subarray sum.
    """
    
    if R in memo:
        return memo[R]
    else:
        # If the right index is less than X-1, the sub array doesn't reach the minimum length X so it is invalid
        if R < X-1:
            answer = -float('inf')
            L[R] = 0
        # If the right index is X-1, there can only be one possible sub array, from index 0 to index X-1. 
        # The sum of the subarray is the value of the prefix-sum array at index X-1 
        elif R == X-1:
            answer = P[R]
            # The left index is 0.
            L[R] = 0
        else:
        # We compare the previous subarray sum from extending the previous subarray with arr[R]
        # versus the new sub array from index R-X+1 to R (this is computed using the prefix-sum array)
            previous_sum = arr[R] + max_subarray_sum(R-1)
            new_sum = P[R] - P[R-X]
            if previous_sum >= new_sum:
                answer = previous_sum
                L[R] = L[R-1]
            else:
                answer = new_sum
                L[R] = R-X+1
    memo[R] = answer            
    return memo[R]

# Input
n = int(input())
X = int(input()) # minimum length of the subarray
arr = [int(x) for x in input().split()]
memo = dict()
L = dict() # the keys are the right indices and the values are the left indices
# We procompute the prefix-sum array of A
P = [arr[0]]
for x in range(1,n):
    P.append(P[x-1] + arr[x])
  
max_subarray_sum(n-1) # fill up the memo table from index 0 to index n-1

max_sum = max(memo.values())
R = max(memo, key = memo.get)

print(f'The max sub array sum is {max_sum} with L = {L[R]} and R = {R}')

print(memo)

 8
 5
   1 -2 -3 10 4 -7 2 -5


The max sub array sum is 10 with L = 0 and R = 4
{4: 10, 5: 3, 6: 6, 7: 4}


## Problem 5

In [93]:
def shuffles(A, B, C):
    m = len(A)
    n = len(B)
    
    # Check if the length of C is not equal to the length of A+B
    
    if m + n != len(C):
        return 0
    
    # Initialize the DP table
    dp = [[0] * (n + 1) for x in range(m + 1)]
    
    # Base Cases
    
    # All the characters in C up to i are from A
    
    for i in range(m+1):
        if A[:i] == C[:i]:
            dp[i][0] = 1
        else:
            dp[i][0] = 0
            
    # All the characters in C up to i are from B
    
    for j in range(n+1):
        if B[:j] == C[:j]:
            dp[0][j] = 1
        else:
            dp[0][j] = 0
    
    
    # Fill up the DP table
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            if A[i-1] == C[i+j-1] or B[j-1] == C[i+j-1]:
                '''
                There are two cases. 
                1. The (i+j)th character of C comes from A
                
                This means that the first i characters of C can be formed in 
                dp[i-1][j] ways using the first i-1 characters of A and all j characters of B.
                
                2. The (i+j)th character of C comes from B
                The first i characters of C can be formed in dp[i][j-1] ways 
                using all i characters of A and the first j-1 characters of B.
                '''
                dp[i][j] = dp[i-1][j] + dp[i][j-1]
            else:
                dp[i][j] = 0
    return dp[m][n]


# Example usage
A = input()
B = input()
C = input()

print(f'The number of ways that C is a shuffle of A and B is {shuffles(A, B, C)}.')




 BANANA
 ANANAS
 BANANAANANAS


The number of ways that C is a shuffle of A and B is 12.
